In [9]:
from src.eda.soporte_eda import *

In [2]:
conectar_bd()

(<connection object at 0x0000015C9A2F5AD0; dsn: 'user=postgres password=xxx dbname=Proyecto_Sephora host=localhost port=5432', closed: 0>,
 <cursor object at 0x0000015C9A29F680; closed: 0>)

En principio se parte de esta query general con los aspectos más relevantes de los productos (aspectos como numero de variaciones o el acabado, textura etc. son caracteristicas consideradas mas secundarias y menos relevantes de los productos que se usaran en el analisis mas especifico para determinar el producto a lanzar pero en el analisis general inicial de Sephora no se consideran). este analisis general inicial tiene como proposito posicionar la marca de sephora collection dentro de sephora, es decir, en comparacion con las otras marcas

In [3]:
query_general = """
select 
    p.url_producto,
    p.nombre,
    p.descripcion,
    m.nombre_marca, 
    c.nombre_categoria,
    s.nombre_subcategoria,
    h.precio,
    h.numero_valoraciones,
    h.valoracion
from productos p 
join marcas m on p.id_marca = m.id_marca
left join categorias c on p.id_categoria = c.id_categoria
left join subcategorias s on p.id_subcategoria = s.id_subcategoria
join historico h on p.id_producto = h.id_producto;
"""

In [35]:
df = ejecutar_query(query_general)
df.head()

,url_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion
0,https://www.sephora.es/p/soft-lit-naturally-lu...,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7
1,https://www.sephora.es/p/triclone-skin-tech-fo...,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5
2,https://www.sephora.es/p/triclone-skin-tech-hy...,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1
3,https://www.sephora.es/p/color-fuse-talc-free-...,Color Fuse Talc-Free Powder Blush With Ferment...,Colorete en polvo,Haus Labs,Rostro,Colorete,32.99,18,4.8
4,https://www.sephora.es/p/bio-radiant-gel-powde...,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,Haus Labs,Rostro,Iluminador,46.99,8,4.8


In [6]:
df.shape

(1829, 9)

In [11]:
def info_reporte(dataframe):
    """
    Genera un informe sobre los valores nulos y tipos de datos de un DataFrame.

    Args:
        dataframe (pd.DataFrame): El DataFrame a analizar.

    Returns:
        pd.DataFrame: Un DataFrame con el número y porcentaje de valores nulos por columna,
                      junto con el tipo de dato de cada columna.
    """
    df_report = pd.DataFrame()
    df_report["Numero_nulos"] = dataframe.isnull().sum()
    df_report["Porcentaje_nulos"] = round((dataframe.isnull().sum()/dataframe.shape[0]*100), 2)
    df_report["Tipo_dato"] = dataframe.dtypes
    return df_report

In [12]:
info_reporte(df)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
url_producto,0,0.00,object
nombre,3,0.16,object
descripcion,98,5.36,object
nombre_marca,0,0.00,object
nombre_categoria,9,0.49,object
nombre_subcategoria,33,1.80,object
precio,0,0.00,float64
numero_valoraciones,0,0.00,int64
valoracion,0,0.00,float64


In [13]:
df["url_producto"].duplicated().sum()

np.int64(7)

In [36]:
df.drop_duplicates(subset=["url_producto"], inplace=True)
df.shape

(1822, 9)

In [24]:
df.head()

,url_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion
0,https://www.sephora.es/p/soft-lit-naturally-lu...,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7
1,https://www.sephora.es/p/triclone-skin-tech-fo...,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5
2,https://www.sephora.es/p/triclone-skin-tech-hy...,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1
3,https://www.sephora.es/p/color-fuse-talc-free-...,Color Fuse Talc-Free Powder Blush With Ferment...,Colorete en polvo,Haus Labs,Rostro,Colorete,32.99,18,4.8
4,https://www.sephora.es/p/bio-radiant-gel-powde...,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,Haus Labs,Rostro,Iluminador,46.99,8,4.8


Me quito los duplicados (comento el problema). y explico que lo hago por el campo de url porque me he dado cuenta de que por los campos principales que habia de los productos en la tabla de productos pues hay productos que si que estan duplicados en nombre, descripcion, marca, categoria y subcategoria aunque verdaderamente no sean los mismos (puedo demostrarlo tambien)

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,1822.0,37.201850,18.895532,3.99,25.0,34.99,45.00,169.0
numero_valoraciones,1822.0,590.726674,1707.459476,0.00,3.0,42.50,406.75,20035.0
valoracion,1822.0,3.799671,1.543679,0.00,3.9,4.40,4.70,5.0


Aqui tengo que hacer un buen analisis de maximos / minimos (outliers). aqui es donde puedo meter algun grafico

In [16]:
df.describe(include="object").T

,count,unique,top,freq
url_producto,1822,1822,https://www.sephora.es/p-640056.html,1
nombre,1819,1646,Moving Lights,14
descripcion,1724,1204,Paleta de sombras de ojos,44
nombre_marca,1822,79,Sephora Collection,198
nombre_categoria,1813,13,Rostro,627
nombre_subcategoria,1789,48,Máscara de pestañas,148


Recuento de productos por marca

In [37]:
df_result = df.groupby("nombre_marca")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
df_result.head(10)

,nombre_marca,url_producto
63,Sephora Collection,198
5,Benefit Cosmetics,86
10,Charlotte Tilbury,77
42,Make Up For Ever,66
15,Dior,65
69,Tarte,64
0,Anastasia Beverly Hills,61
66,Sisley,58
21,Fenty Beauty,58
39,Lancôme,57


Recuento de productos por categoria 

In [39]:
df_result = df.groupby("nombre_categoria")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
df_result.head(10)

,nombre_categoria,url_producto
10,Rostro,627
4,Ojos,372
3,Labios,343
8,Pinceles & accesorios,167
0,Cejas,100
1,Desmaquillante,58
5,Paletas maquillaje,49
12,Uñas,43
2,Estuches maquillaje,42
11,Sacapuntas,4


Recuento de procuctos por subcategoria 

In [40]:
df_result = df.groupby("nombre_subcategoria")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
df_result.head(10)

,nombre_subcategoria,url_producto
32,Máscara de pestañas,148
2,Barra de labios,126
8,Colorete,113
4,Base de maquillaje,107
42,Primer & spray fijador,104
6,Brocha rostro,103
1,Antiojeras & corrector,80
25,Gloss/ Brillo labial,79
44,Sombras de ojos,76
34,Paleta de sombras de ojos,74
